In [ ]:
import cv2
import numpy as np
from PIL import ImageFont
import matplotlib.pyplot as plt

# multiprocessing
import multiprocessing as mp

In [ ]:
# load font
# font = ImageFont.truetype('/usr/share/fonts/truetype/Sarai/Sarai.ttf', 24)
font = ImageFont.truetype('./PingFang.ttc', 24)
additional_char = ''

In [ ]:
# Load special characters
additional_char = '┌┍┎┏┐┑┒┓└┕┖┗┘┙┚┛├┝┞┟┠┡┢┣┤┥┦┧┨┩┪┫┬┭┮┯┰┱┲┳┴┵┶┷┸┹┺┻┼┽┾┿╀╁╂╃╄╅╆╇╈╉╊╋╌╍╎╏═║╒╓╔╕╖╗╘╙╚╛╜╝╞╟╠╡╢╣╤╥╦╧╨╩╪╫╬◤◥◄►▶◀◣◢▲▼◥▸◂▴▾△▽▷◁⊿▻◅▵▿▹◃❏❐❑❒▀▁▂▃▄▅▆▇▉▊▋█▌▍▎▏▐░▒▓▔▕■□▢▣▤▥▦▧▨▩▪▫▬▭▮▯'

In [ ]:
# Pre-process font
def get_maxheight():
    maxheight = 0
    for i in range(32,127):
        char = chr(i)
        maxheight=max(maxheight,font.getsize(char)[1])
    for char in additional_char:
        maxheight=max(maxheight,font.getsize(char)[1])
    return maxheight

def get_maxwidth():
    maxwidth = 0
    for i in range(32,127):
        char = chr(i)
        maxwidth=max(maxwidth,font.getsize(char)[0])
    for char in additional_char:
        maxwidth=max(maxwidth,font.getsize(char)[0])
    return maxwidth

# build font dict
maxheight = get_maxheight()
maxwidth = get_maxwidth()
# print(maxwidth)

# full
dictionary_bitmap = []
dictionary_char = []
dictionary_ord = {}
for i in range(32,127):
    char = chr(i)
    print(char,end='')
    img, bias = font.getmask2(char)
    bitmap = np.zeros((maxheight,font.getsize(char)[0]))
    img = np.array(img).reshape(img.size[1],img.size[0])
    if bias[0]>=0:
        bitmap[bias[1]:bias[1]+img.shape[0],bias[0]:] = img
    else:
        bitmap[bias[1]:bias[1]+img.shape[0],:] = img[:,-bias[0]:]
    dictionary_bitmap.append(255-bitmap)
    dictionary_char.append(char)
    dictionary_ord[char]=len(dictionary_bitmap)-1
# print(dictionary_char)
for char in additional_char:
    i = ord(char)
    print(char,end='')
    img, bias = font.getmask2(char)
    bitmap = np.zeros((maxheight,font.getsize(char)[0]))
    img = np.array(img).reshape(img.size[1],img.size[0])
    if bias[0]>=0:
        bitmap[bias[1]:bias[1]+img.shape[0],bias[0]:] = img
    else:
        bitmap[bias[1]:bias[1]+img.shape[0],:] = img[:,-bias[0]:]
    dictionary_bitmap.append(255-bitmap)
    dictionary_char.append(char)
    dictionary_ord[char]=len(dictionary_bitmap)-1

In [ ]:
def row2str(row):
    # padding
    row_width = row.shape[1]
    for _ in range(maxwidth):
        row = np.hstack([row,row[:,-1:]])
    
    row_pos = 0
    row_text = ''
    while row_pos < row_width:
        candidates = np.zeros(len(dictionary_bitmap))
        for cidx, char in enumerate(dictionary_bitmap):
            row_part = row[:,row_pos:row_pos+char.shape[1]]
            loss = np.square(np.abs(row_part-char)).mean()
            candidates[cidx]= loss
        cidx = candidates.argmin()
        row_text += dictionary_char[cidx]
        row_pos += dictionary_bitmap[cidx].shape[1]
    return row_text

# Video

In [ ]:
targrt_rows = 32
frame_skip = 1
if __name__ == '__main__':
    cap = cv2.VideoCapture('./bad apple原版高清1440*1080.mp4')
    ret, src = cap.read()
    src_scale = targrt_rows*maxheight/src.shape[0]
    writer = cv2.VideoWriter(
        './bad apple.mp4',
        cv2.VideoWriter_fourcc('F', 'M', 'P', '4'),
        cap.get(cv2.CAP_PROP_FPS)/frame_skip,
        (int(src_scale*src.shape[1]),int(src_scale*src.shape[0])),
    )
    print('size',(int(src_scale*src.shape[1]),int(src_scale*src.shape[0])))
    frame_cnt = 1
    frame_tot = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    while True:
        print('% 6d % 6d %.2f'%(frame_cnt, frame_tot, frame_cnt*100.0/frame_tot),end='%\r')
            
        # pre-process image
        src = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
        src_scale = targrt_rows*maxheight/src.shape[0]
        src = cv2.resize(src,(int(src_scale*src.shape[1]),int(src_scale*src.shape[0])))
#         ret, src = cv2.threshold(src, 127, 255, cv2.THRESH_BINARY)
        

        # prepare rows
        rows = []
        for i in range(targrt_rows):
            rows.append(src[i*maxheight:(i+1)*maxheight,:])

        # generate image
        final_image = 255 - np.zeros((src.shape[0],src.shape[1]+maxwidth), dtype=np.uint8)
        x = y =0
        for row in list(pool.map(row2str, rows)):
            y=0
            for c in row:
                bitmap = dictionary_bitmap[dictionary_ord[c]]
                final_image[x:x+maxheight,y:y+bitmap.shape[1]]=bitmap
                y+=bitmap.shape[1]
            x += maxheight
        final_image = final_image[:,:src.shape[1]]
        final_image = np.array([final_image,final_image,final_image]).transpose(1,2,0)
        writer.write(final_image)
        for _ in range(frame_skip):
            ret, src = cap.read()
        frame_cnt += frame_skip
        if src is None:
            break
    writer.release()

# Image

In [ ]:
targrt_rows = 48

pool = mp.Pool()
# cap = cv2.VideoCapture('./bad apple原版高清1440*1080.mp4')
# for _ in range(150):
#     ret, src = cap.read()
src = cv2.imread('./32470225.png')
src_scale = targrt_rows*maxheight/src.shape[0]

# pre-process image
src = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
src_scale = targrt_rows*maxheight/src.shape[0]
src = cv2.resize(src,(int(src_scale*src.shape[1]),int(src_scale*src.shape[0])))
#         ret, src = cv2.threshold(src, 127, 255, cv2.THRESH_BINARY)


# prepare rows
rows = []
for i in range(targrt_rows):
    rows.append(src[i*maxheight:(i+1)*maxheight,:])

# generate image
final_image = 255 - np.zeros((src.shape[0],src.shape[1]+maxwidth), dtype=np.uint8)
x = y =0
for row in list(pool.map(row2str, rows)):
    print(row)
    y=0
    for c in row:
        bitmap = dictionary_bitmap[dictionary_ord[c]]
        final_image[x:x+maxheight,y:y+bitmap.shape[1]]=bitmap
        y+=bitmap.shape[1]
    x += maxheight
final_image = final_image[:,:src.shape[1]]
final_image = np.array([final_image,final_image,final_image]).transpose(1,2,0)

In [ ]:
cv2.imwrite('output.png', final_image)